# A problem of black and brown mice

Solve the Box and Tao problem of parental probabilities for black and
brown mice. See the text for full explanation.

In [ ]:
import numpy as np

rnd = np.random.default_rng()

n_trials = 100_000

# Make an array to store results for each trial.
# The results are strings, so use dtype=object.
# Many of these we will not set, for example, for brown mice (see below).
parents = np.zeros(n_trials, dtype=object)

for i in range(n_trials):
    test_mouse = rnd.choice(['BB', 'Bb', 'bb'], p=[0.25, 0.5, 0.25])

    # The test mouse is black; since we drew a brown mouse skip this trial
    if test_mouse == 'bb':
        # continue has the effect of aborting this iteration of the loop
        # and going back to start the next iteration.  If the code gets
        # to "continue", none of the rest of the loop (indented) code
        # will run.
        continue

    # If the test mouse is 'BB', all 7 children are guaranteed to
    # be 'Bb' black.
    # Therefore, add 'BB' to the parent list.
    if test_mouse == 'BB':
        parents[i] = 'BB'

    # If the parent mouse is 'Bb', we draw 7 children to
    # see whether all of them are black ('Bb').
    # The probabilities come from the middle row of the table.
    if test_mouse == 'Bb':
        children = rnd.choice(['Bb', 'bb'], p=[0.5, 0.5], size=7)
        if np.all(children == 'Bb'):
            parents[i] = 'Bb'

# Now, count how many parents were 'BB' vs 'Bb'
n_parents_BB = np.sum(parents == 'BB')
n_parents_Bb = np.sum(parents == 'Bb')
n_B = n_parents_BB + n_parents_Bb

p_BB = n_parents_BB / n_B
p_Bb = n_parents_Bb / n_B

print('p_BB =', np.round(p_BB, 3))

In [ ]:
print('p_Bb =', np.round(p_Bb, 3))

In [ ]:
print('Ratio =', np.round(p_BB / p_Bb, 1))

We see that all the offspring being black considerably changes the
situation! We started with the odds being 2:1 in favor of Bb vs BB. The
“posterior” or “after the evidence” ratio is closer to 64:1 in favor of
*BB*! (1973, pp. 12-14)

Let’s tune the code a bit to run faster. Instead of doing the trials one
mouse at a time, we will do the whole bunch together.

To do this, we will use two-dimensional arrays.

So far, nearly all the arrays we have used are one-dimensional. A
one-dimensional array is a sequence of values. Let us generate a
one-dimensional array with `rnd.choice`, as we have many times in this
book, and in this chapter.

In [ ]:
# A one-dimensional array, with five elements.
one_d = rnd.choice([1, 2], size=5)
one_d

However, we can also generate arrays with more than one dimension. In
particular we can generate arrays with two dimensions. An array with two
dimensions has rows and columns, much like a Pandas data frame. However,
unlike data frames, two-dimensional arrays have no row or column names.
Here is a two-dimensional array we create with `rnd.choice`, by passing
two values to the size argument:

In [ ]:
# A two-dimensional array with five rows and three columns.
two_d = rnd.choice([1, 2], size=(5, 3))
two_d

As usual, we can apply Boolean comparison operations to this array, to
get a two-dimensional Boolean array:

In [ ]:
is_2 = two_d == 2
is_2

Numpy thinks of two-dimensional arrays as having two *axes*, where the
first axis (axis at position 0) is the row axis, and the second axis (at
position 1) is the column axis.

Many Numpy functions have an `axis` argument that asks the function to
apply its operation along a particular axis. For example, we might want
to ask whether `all` the values in *each column* (across axis
position 1) are equal to 2. We can do this using `np.all`:

In [ ]:
all_equal_2 = np.all(is_2, axis=1)
all_equal_2

Notice that we get one answer for each row (axis=0), where the answer is
`np.all` across the columns, for that row.

In [ ]:
n_trials = 1_000_000

# In n_trials trials, pair two Bb mice and generate a child.
test_mice = rnd.choice(['BB', 'Bb', 'bb'], p=[0.25, 0.5, 0.25], size=n_trials)

# The resulting test mouse is black, so filter out all brown ones.
test_mice = test_mice[test_mice != 'bb']
n_test_mice = len(test_mice)

# Each test mouse will now be mated with a brown mouse, producing 7 offspring.
# We then store whether all the offspring were black or not.
all_offspring_black = np.zeros(n_test_mice, dtype=bool)

# If a test mouse is 'BB', we are assured that all its offspring
# will be black.
all_offspring_black[test_mice == 'BB'] = True

# If a test mouse is 'Bb', we have to generate its offspring and
# see whether they are all black or not.
test_mice_Bb = (test_mice == 'Bb')
n_test_mice_Bb = np.sum(test_mice_Bb)

# Generate all offspring of all 'Bb' test mice.
# This gives a 2-dimensional array, with one row per Bb mouse,
# and 7 columns, one for each child.
offspring = rnd.choice(
    ['Bb', 'bb'], p=[0.5, 0.5], size=(n_test_mice_Bb, 7)
)
# Check whether all children (columns) are Bb, for each row.
all_offspring_black[test_mice_Bb] = np.all(offspring == 'Bb', axis=1)

# Find the genetic types of the parents of all-black offspring.
parents = test_mice[all_offspring_black]

# Calculate what fraction of parents were 'BB' vs 'Bb'.
parents_BB = (parents == 'BB')
parents_Bb = (parents == 'Bb')
n_B = np.sum(all_offspring_black)

p_BB = np.sum(parents_BB) / n_B
p_Bb = np.sum(parents_Bb) / n_B

print('p_BB = ', np.round(p_BB, 3))

In [ ]:
print('p_Bb = ', np.round(p_Bb, 3))

In [ ]:
print('Ratio = ', np.round(p_BB / p_Bb, 1))

This yields a similar result, but in much shorter time — which means we
can increase the number of trials and get a more accurate result.